In [1]:
from urllib.request import urlopen
url = "http://www.baidu.com"
resp=urlopen(url)
data=resp.read().decode("utf-8")
with open("mybaidu1.html",mode='w',encoding="utf-8") as f:  #使用 with 关键字系统会自动调用 f.close() 方法， with 的作用等效于 try/finally 语句是一样的。
    f.write(data)
    f.close()
print("over")
resp.close()

over


# web请求过程
+ 服务器渲染：在服务器端将数据和html整合在一起，统一返回给客户端
+ 客户端渲染：第一次请求返回html骨架（页面源代码看不到数据），第二次请求拿到数据，在客户端进行html和数据的整合。  
在爬取网页前要判定其为哪种请求方式。  

# HTTP协议
协议：两台计算机之间进行数据传输的设置的一个协定，常见的有HTTP,TCP/IP,SMTP...  
HTTP:"Hyper Text Transfer Protocol",超文本传输协议,用于浏览器和服务器之间的数据交互。
HTTP协议将一条信分为三大块内容，无论是请求还是响应。  
请求：
```
请求行：请求方式(get/post),请求的URL地址 协议
请求头：放一些服务器要使用的附加信息：如对数据的要求
请求体：放一些请求参数，如搜索内容
```
响应:
```
状态行：协议，状态码
响应头：放一些客户端需要使用的附加信息：如cookie
响应体：返回客户端真正需要的内容
```
user-agent:请求信息的身份标识（请求用什么发送）  
referer：防盗链（请求从哪里来）  
cookie：本地字符串数据信息(用户登录信息)  
请求方式：   
GET:查询，显示提交   
POST：修改提交，隐示提交

In [1]:
import requests
url="http://www.sogou.com/web?query=周杰伦"
resp = requests.get(url) #采用get请求
print(resp)
with open("sougoujay1.html",mode='w',encoding="utf-8") as f:
    f.write(resp.text)
    print("over")
resp.close()
print("--------------------------------------------")
headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML,like Gecko) Chrome/102.0.5005.63 Safari/537.36 Edg/102.0.1245.30"}
resp2=requests.get(url,headers=headers)
print(resp2)
with open("sougoujay2.html",mode='w',encoding="utf-8") as f:
    f.write(resp2.text)
    print("over")
resp2.close()

<Response [200]>
over
--------------------------------------------
<Response [200]>
over


In [9]:
import requests
query=input("输入你喜欢的明星:")
url='http://www.sogou.com/web?query='+query
headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML,like Gecko) Chrome/102.0.5005.63 Safari/537.36 Edg/102.0.1245.30"}
resp3=requests.get(url,headers=headers)
print(resp3)
data=resp3.text
with open("sougoustar.html",mode='w',encoding='utf-8') as f:
    f.write(data)
    print("over")
resp3.close()

输入你喜欢的明星:周杰伦
<Response [200]>
over


In [11]:
import requests
url="https://fanyi.baidu.com/sug"
s=input("请输入你要查的单词:")
data={"kw":s}#发送的数据必须放在字典中
resp=requests.post(url,data=data)
print(resp.json()) #将服务器返回的内容直接处理成json，实际上是转换成python字典形式

请输入你要查的单词:hero
{'errno': 0, 'data': [{'k': 'hero', 'v': 'n. 英雄，勇士; 男主角; （古代神话中的）神人，半神的勇士'}, {'k': 'Hero', 'v': '[人名] 海罗; [电影]英雄'}, {'k': 'HERO', 'v': 'abbr. Hazard of Electromag netic Radiation of Ordn'}, {'k': 'Herod', 'v': 'n. 希律王（约公元前73-公元前4）'}, {'k': 'heron', 'v': 'n. <动>鹭'}]}


In [27]:
import requests
import json
url="https://movie.douban.com/j/chart/top_list"
params={
    "type": "24",
    "interval_id": "100:90",
    "action": "",
    "start": 0,
    "limit": 20
}
resp=requests.get(url,params=params)
print(resp.request.url)
with open("douban1.html",mode='w',encoding='utf-8') as f:
    f.write(resp.text)#被反爬
    print("over")
print(resp.request.headers)#查看用户，请求从哪里发送
headers={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36 Edg/102.0.1245.30"}
resp=requests.get(url,params=params,headers=headers)
with open("douban2.txt",mode='w',encoding='utf-8') as f:
    f.write(resp.text)
    print("over")
resp.close()

https://movie.douban.com/j/chart/top_list?type=24&interval_id=100%3A90&action=&start=0&limit=20
over
{'User-Agent': 'python-requests/2.26.0', 'Accept-Encoding': 'gzip, deflate, br', 'Accept': '*/*', 'Connection': 'keep-alive'}
over


# 数据解析
+ re解析
+ bs4解析
+ xpath解析

## re解析
regular expression，正则表达式，一种使用表达式的方式对字符串进行匹配的语法规则  
正则的语法：使用元字符进行排列组合来匹配字符串。
每个元字符默认只匹配一个字符
```
    .:匹配除换行符以外的任意单个字符
    \w:匹配数字/字母/下划线
    \W:匹配非数字/字母/下划线
    \s:匹配任意空白符
    \S:匹配非任意空白符
    \d:匹配数字
    \D:匹配非数字
    \n:匹配一个换行符
    \t:匹配一个制表符
    ^:匹配字符串的开始
    $:匹配字符串的结尾
    a|b:匹配字符a或者字符b
    ():匹配括号内的表达式
    [...]:匹配到中括号中的字符，如[a-zA-Z0-9_]相当于\w
    [^...]:匹配到除了中括号中的所有字符，如[^a-zA-Z0-9_]相当于\W
```
量词
```
    *：重复0次或者更多次
    +：重复1次或者更多次
    ？：重复0次或者1次
   {n}:重复n次
   {n,}:重复n次或者更多次
   {n,m}:重复n到m次
   .*？：惰性匹配 ，尽可能少的匹配  
   .*：贪婪匹配，尽可能多的匹配
```

### re解析实战

In [29]:
import re
list1=re.findall("\d+","my phone number:10010,100896")
print(list1)
print("-----------")
it=re.finditer("\d+","my phone number:10010,100896")#返回一个迭代器对象
print(it)
for i in it:
    print(i)
    print(i.group())
print("-----------")
s=re.search("\d+","my phone number:10010,100896")#search找到一个结果就返回，返回结果是match对象
print(s)
print(s.group())
print("-----------")
m=re.match("\d+","10010,100896")#从头开始匹配,相当于"^\d+",找到一个结果就返回
print(m)
print(m.group())
print(m.span())
print("-----------")
#预加载正则表达式
obj=re.compile("\d+")
ret=obj.finditer("my phone number:10010,100896")
for i in ret:
    print(i.group())

['10010', '100896']
-----------
<re.Match object; span=(16, 21), match='10010'>
10010
<re.Match object; span=(22, 28), match='100896'>
100896
-----------
<re.Match object; span=(16, 21), match='10010'>
10010
-----------
<re.Match object; span=(0, 5), match='10010'>
10010
(0, 5)
-----------
10010
100896


In [1]:
import re
#Python中三引号可以将复杂的字符串进行赋值，允许一个字符串跨多行，字符串中可以包含换行符、制表符以及其他特殊字符。
s="""
<div class='jay'><span id='1'>周杰伦</span></div>
<div class='jj'><span id='2'>林俊杰</span></div>
<div class='jolin'><span id='3'>范思哲</span></div>
<div class='sylar'><span id='4'>邓紫棋</span></div>
<div class='tory'><span id='5'>胡歌</span></div>
"""
obj=re.compile("<div class='.*?'><span id='\d+'>.*?</span></div>")
iter1=obj.finditer(s)
for i in iter1:
    print(i)
#(?P<key>正则表达式)将值赋给key，可通过group访问
obj1=re.compile("<div class='(?P<class>.*?)'><span id='(?P<id>\d+)'>(?P<name>.*?)</span></div>")
iter2=obj1.finditer(s)
for i in iter2:
    print(i.group("class"),end="  ")
    print(i.group("id"),end="  ")
    print(i.group("name"))   

<re.Match object; span=(1, 47), match="<div class='jay'><span id='1'>周杰伦</span></div>">
<re.Match object; span=(48, 93), match="<div class='jj'><span id='2'>林俊杰</span></div>">
<re.Match object; span=(94, 142), match="<div class='jolin'><span id='3'>范思哲</span></div>">
<re.Match object; span=(143, 191), match="<div class='sylar'><span id='4'>邓紫棋</span></div>">
<re.Match object; span=(192, 238), match="<div class='tory'><span id='5'>胡歌</span></div>">
jay  1  周杰伦
jj  2  林俊杰
jolin  3  范思哲
sylar  4  邓紫棋
tory  5  胡歌


#### 爬取豆瓣top电影

In [31]:
import requests
import re
import csv
url="https://movie.douban.com/chart"
headers={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36 Edg/102.0.1245.30"}
resp=requests.get(url,headers=headers)
data=resp.text
obj=re.compile('<table width="100%" class="">.*?title="(?P<name>.*?)">.*?<p class="pl">(?P<time>.*?)/.*?<span class="rating_nums">(?P<grade>.*?)</span>.*?<span class="pl">(?P<people>.*?)</span>',re.S)#re.S让.能匹配到换行符
iter1=obj.finditer(data)
f=open("豆瓣top电影.csv",'w',newline="")
f1=csv.writer(f)
for i in iter1:
    f1.writerow(i.groupdict().values())   
print("over")
f.close()
resp.close()

over


#### 爬取盗影天堂影片下载地址

In [69]:
import requests
import re
import csv
url1="http://dytt89.com/"
header={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36 Edg/102.0.1245.33"}
resp1=requests.get(url1,headers=headers)
resp1.encoding='gbk'
data=resp1.text
with open("盗版天堂.html",mode='w',encoding='gbk') as f:
    f.write(data)
    print("over")
obj=re.compile('2022必看热片.*?<ul>(?P<ul>.*?)</ul>',re.S)
iter1=obj.finditer(data)
for i in iter1:
    data2=i.group("ul")
obj1=re.compile("<a href='(?P<herf>.*?)'.*?>(?P<title>.*?)</a>",re.S)
iter2=obj1.finditer(data2)
f=open("2022热片及下载地址.csv",mode='w',newline="")
f1=csv.writer(f)
for i in iter2:
    url2="http://dytt89.com/"+i.group('herf').strip("/")
    resp2=requests.get(url2,headers=headers)
    resp2.encoding='gbk'
    data3=resp2.text
    obj2=re.compile('<td style="WORD-WRAP: break-word" bgcolor="#fdfddf"><a href="(?P<download>.*?)"',re.S)
    ret=obj2.search(data3)
    list1=[]
    list1.append(i.group("title"))
    list1.append(ret.group("download"))
    f1.writerow(list1)
    resp2.close()
f.close()
resp1.close()
#re.match() 从头匹配一个符合规则的字符串，从起始位置开始匹配，匹配成功返回一个对象，未匹配成功返回None。
#re.search()浏览全部字符串，匹配第一符合规则的字符串，浏览整个字符串去匹配第一个，未匹配成功返回None。
#如果字符串开头就与关键字不匹配，那么就无法使用.match()方法。
#如果一个字符串中有多个符合匹配规则的配置项，想获得第二个匹配结果，那么就无法是使用.search()方法。

over


## bs4解析
BeautifulSoup，通过html标签和属性去定位页面上的内容  
1.从bs4中导入beautifulsoup函数  
2.将获取的页面源代码交给beautifulsoup处理，生成bs4对象  
3.调用其内部地find方法或者find_all方法，以标签和属性为参数一步步缩小范围  
4.再通过text方法获得其标签内的内容

### bs4解析实战

#### 北京新发地菜价爬取蔬菜前5页内容
由于其网页源代码的重构所以这里不能用bs4解析，因此这里我们采用re解析进行处理

In [69]:
import requests
import re
import csv
url="http://www.xinfadi.com.cn/getPriceData.html"
headers={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36 Edg/102.0.1245.33"}
params={
    "limit": 20,
    "current":"", 
    "pubDateStartTime":"",
    "pubDateEndTime": "",
    "prodPcatid": "",
    "prodCatid": "",
    "prodName": ""
}
f=open("新发地蔬菜.csv",mode='w',newline="")
f1=csv.writer(f)
obj=re.compile('prodName":"(?P<name>.*?)".*?"lowPrice":"(?P<lowPrice>.*?)","highPrice":"(?P<highPrice>.*?)","avgPrice":"(?P<avgPrice>.*?)","place":"(?P<place>.*?)","specInfo":"(?P<specInfo>.*?)","unitInfo":"(?P<unitInfo>.*?)","pubDate":"(?P<pubDate>.*?)"',re.S)
for i in range(1,6):
    params['current']=i
    resp=requests.post(url,headers=headers,params=params)
    data=resp.text
    iter1=obj.finditer(data)
    for i in iter1:
        f1.writerow(i.groupdict().values())
    print("over")
f.close()
resp.close()

over
over
over
over
over


#### 抓取优美图库图片

In [8]:
import requests
from bs4 import BeautifulSoup
import time
url="https://www.umei.cc/bizhitupian/weimeibizhi/"
headers={"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36 Edg/102.0.1245.33"}
resp=requests.get(url,headers=headers)
resp.encoding='utf-8'
main_page=BeautifulSoup(resp.text,"html.parser")
alist=main_page.find("ul",class_="pic-list after").find_all("a")
for i in alist[0:10]:
    herf=i.get("href")
    child_page_resp=requests.get("https://www.umei.cc"+herf,headers=headers)
    child_page_resp.encoding='utf-8'
    child_page=BeautifulSoup(child_page_resp.text,"html.parser")
    section=child_page.find("section",class_="img-content")
    img=section.find("img")
    img_url=img.get("src")
    img_resp=requests.get(img_url,headers=headers)
    img_name=img_url.split("/")[-1]
    with open("../Photo/"+img_name,mode='wb') as f:
        f.write(img_resp.content)#拿到该图片的字节形式
    print("over",img_name)
    img_resp.close()
    time.sleep(1)

print("all over")
resp.close()

over 9e5827678bd12db0999a573254e40d1e.jpg
over 718c092b7239a6c185c2a0fc3167bef8.jpg
over 26291c282da777b06d9ca389cad82d7f.jpg
over d52d1b3880854b7963b72eab6234b53e.jpg
over e5d9c2e8a189059e61f6f757aedb79e4.jpg
over 7fe975add2196501191db41a6dddfe21.jpg
over ba34fff6a58897cc5362b79e477c06d8.jpg
over d2c8044b3aad7678456206a7787632f6.jpg
over 83db2e50ceeab45aae2e4121e3b9ed86.jpg
over b66e5a43535d333dc83b5ea5f5256115.jpg
all over


## xpath解析
xpath是XML文档搜索内容的一门语言  
html是xml的一个子集

### xpath实战

#### 爬取猪八戒网

In [23]:
import requests 
from lxml import etree
import csv
url="https://xa.58.com/ershoufang/"
headers={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.81 Safari/537.36 Edg/104.0.1293.54",
        }
resp=requests.get(url,headers=headers)
#print(resp.text)
html=etree.HTML(resp.text) # 加载html代码
a=html.xpath("/html/body/div[1]/div/div/section/section[3]/section[1]/section[2]/div")
for i in a:
    d=i.xpath("./a/div[2]/div[1]/div[1]/h3/text()")
    print(d)


['急售大寨路融侨城四期187平精装四室高楼层不临街满二看房方便']
['紫园sunny 电梯房精装修双气带中央空调朝南证在手可按揭']
['首付20个精装现房 带车位 中间楼层 不限户 高绿化五证齐全']
['西安精装修交付   旁边的房子 智轨交通 南北通透 商圈成熟']
['纺织城地铁口 铁一中旁 精装修13500 超高性价比']
['邮电十所家属院 140平 南北通透 电梯房 诚心出售 价格可']
['南北通透 精装修 铭翔花园 满五  住房 采光好 周边商圈成']
['业主诚心出售 体育场地铁口 长安大街3号 两室一厅 随时看房']
['大兴东路 老城根对面 御笔华庭精装修两居室 满两年 无增值税']
['沣西地铁口 吾悦广场旁 业主急售 中交长安里 香榭汇丰']
['宝安紫韵139南北通透3室2厅2卫毛坯现房满二95万议']
['（231w融侨城西区145平）金光门地铁口 南北通透 好楼层']
['西咸新区北客站北 特价河景房9800元 精装修三室两卫三阳台']
['龙记观园124毛坯现房  3室2厅2卫双阳台  90万']
['258萬可谈 147平 满二 国润城(一期) 精装修 电梯房']
['星火路 浩林方里 精装两室 视野采光好 中间楼层 满二 急售']
['西安国际美术城现房别墅  后一套联排 看房方便  专车接送']
['地铁加持奥体板块 央企华润置业紫云府 纵享浐灞生态一线环境']
['凤城四路 华远枫悦 三室毛坯 南北通透 诚心出售 可自行装修']
['福源二期电梯6楼花园洋房，1364室，送屋顶花园']
['西安北 地铁口 总高11层准现房 大赠送 单价10900']
['赠送40平米阁楼 层高6米 南北3居室 送20平米阳台急售']
['龙湖紫都城 3室2厅 精装修 满五  住房 居家自住 保养好']
['保利心语花园 顶楼带露台 赠送100左右 俯视整个小区景观']
['房源采光好，南北通透，业主诚意出']
['清盘特价） 现房改善 一线河景大平层 价格可谈 看房可接送']
['南北通透，业主诚意出，双卧朝南 全天采光 急售 看房随时']
['会展中心，后海公园，大四室仅185W']
['汇通太古城经典四室全明户型 诚心出售 随时看房']
['电梯房 93平 锦绣天下(一期) 2室2厅 南北通透']
['10号地铁口 精装三室 单价

In [2]:
import requests 
from lxml import etree
import csv
url="https://www.aqistudy.cn/historydata/"
headers={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.81 Safari/537.36 Edg/104.0.1293.54",
        }
resp=requests.get(url,headers=headers)
#print(resp.text)
html=etree.HTML(resp.text) # 加载html代码
a=html.xpath("/html/body/div[3]/div/div[1]/div[2]/div[2]/ul")
for i in a:
    d=i.xpath("./div[2]/li")
    for j in d:
        c=j.xpath("./a/text()")
        print(c)

['阿坝州']
['安康']
['阿克苏地区']
['阿里地区']
['阿拉善盟']
['阿勒泰地区']
['安庆']
['安顺']
['鞍山']
['克孜勒苏州']
['安阳']
['蚌埠']
['白城']
['保定']
['北海']
['宝鸡']
['北京']
['毕节']
['博州']
['白山']
['百色']
['保山']
['白沙']
['包头']
['保亭']
['本溪']
['巴彦淖尔']
['白银']
['巴中']
['滨州']
['亳州']
['长春']
['昌都']
['常德']
['成都']
['承德']
['赤峰']
['昌吉州']
['五家渠']
['昌江']
['澄迈']
['重庆']
['长沙']
['常熟']
['楚雄州']
['朝阳']
['沧州']
['长治']
['常州']
['潮州']
['郴州']
['池州']
['崇左']
['滁州']
['定安']
['丹东']
['东方']
['东莞']
['德宏州']
['大理州']
['大连']
['大庆']
['大同']
['定西']
['大兴安岭地区']
['德阳']
['东营']
['黔南州']
['达州']
['德州']
['儋州']
['鄂尔多斯']
['恩施州']
['鄂州']
['防城港']
['佛山']
['抚顺']
['阜新']
['阜阳']
['富阳']
['抚州']
['福州']
['广安']
['贵港']
['桂林']
['果洛州']
['甘南州']
['固原']
['广元']
['贵阳']
['甘孜州']
['赣州']
['广州']
['淮安']
['海北州']
['鹤壁']
['淮北']
['河池']
['海东地区']
['邯郸']
['哈尔滨']
['合肥']
['鹤岗']
['黄冈']
['黑河']
['红河州']
['怀化']
['呼和浩特']
['海口']
['呼伦贝尔']
['葫芦岛']
['哈密地区']
['海门']
['海南州']
['淮南']
['黄南州']
['衡水']
['黄山']
['黄石']
['和田地区']
['海西州']
['河源']
['衡阳']
['汉中']
['杭州']
['菏泽']
['贺州']
['湖州']
['惠州']
['吉安']
['金昌']
['晋城']
['景德镇']
['金华']
['西双版纳州']
[

In [33]:
import requests 
from lxml import etree
import csv
url="https://nanyang.zbj.com/search/f/?kw=saas&r=1"
headers={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.81 Safari/537.36 Edg/104.0.1293.54",
        }
resp=requests.get(url,headers=headers)
html=etree.HTML(resp.text) # 加载html代码

div_list=html.xpath("/html/body/div[2]/div/div/div[2]/div/div[3]/div[4]/div[1]/div[1]/div[3]/a/text()") #拿到该xpath下的节点列表,下标从1开始
print(div_list)


[]


## cookie处理登录17k小说网

In [83]:
import requests
session=requests.session()
data={
    "loginName":"", #写自己的
    "password":""   #写自己的
}
url="https://passport.17k.com/ck/user/login"
resp=session.post(url,data=data)#创建一个会话，该会话就相当于一串连续的请求，在这个过程中cookie不会丢失,data和params参数是一致的，data不会改变url，params会
print(session.cookies)
#以上使会话获得cookie
url1="https://user.17k.com/ck/author/shelf?page=1&appKey=2406394919"
resp1=session.get(url1)
for i in resp1.json()['data']:
    print(i.get('bookName')) #新用户默认读取到自动推荐的书

<RequestsCookieJar[<Cookie accessToken=avatarUrl%3Dhttps%253A%252F%252Fcdn.static.17k.com%252Fuser%252Favatar%252F08%252F08%252F49%252F96964908.jpg-88x88%253Fv%253D1654606766000%26id%3D96964908%26nickname%3D%25E4%25B9%25A6%25E5%258F%258Bx579304A2%26e%3D1670162135%26s%3Dcf661eefeecb3669 for .17k.com/>, <Cookie c_channel=0 for .17k.com/>, <Cookie c_csc=web for .17k.com/>, <Cookie uuid=7378A2BB-C2EE-E80B-6B4F-CA4C3B0B1273 for .17k.com/>]>
我七个姐姐绝世无双
我在菜市口斩妖除魔那些年
新婚夜，我凭孕吐火爆娱乐圈
震惊！我的徒弟居然是女帝
选秀综艺后，玄学大佬制霸娱乐圈


In [1]:
import requests
url="https://user.17k.com/ck/author/shelf?page=1&appKey=2406394919"
resp=requests.get(url,headers={"Cookie":""}#写自己的cookie
for i in resp.json()['data']:
    print(i.get('bookName'))

我七个姐姐绝世无双
我在菜市口斩妖除魔那些年
新婚夜，我凭孕吐火爆娱乐圈
震惊！我的徒弟居然是女帝
选秀综艺后，玄学大佬制霸娱乐圈


## 防盗链的处理
### 爬取梨视频

In [19]:
import requests
url="https://www.pearvideo.com/video_1764627"
cont_id=url.split("_")[-1]
url1="https://www.pearvideo.com/videoStatus.jsp?contId=1764627&mrd=0.7890608429828905"
resp = requests.get(url1)
print(resp.json()) #触发反爬机制
resp.close()
headers ={
    "Referer": "https://www.pearvideo.com/video_1764627"
}
resp1 = requests.get(url1,headers=headers)
url2=resp1.json()['videoInfo']['videos']['srcUrl']
system_time=resp1.json()['systemTime']
print(url2)
video_url=url2.replace(system_time,f"cont-{cont_id}") #f-string的大括号 {} 可以填入表达式或调用函数，Python会求出其结果并填入返回的字符串内：
print(video_url)
resp1.close()
resp2=requests.get(video_url)
with open("梨视频.mp4",mode='wb') as f:
    f.write(resp2.content)
    print("over")
resp2.close()

{'resultCode': '5', 'resultMsg': '该文章已经下线！', 'systemTime': '1654654058133'}
https://video.pearvideo.com/mp4/adshort/20220607/1654654058324-15892168_adpkg-ad_hd.mp4
https://video.pearvideo.com/mp4/adshort/20220607/cont-1764627-15892168_adpkg-ad_hd.mp4
over


### 爬取全国疫情数据(含港澳台)

In [4]:
import requests
import pandas as pd
headers={"Referer": "https://news.qq.com/",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36 Edg/102.0.1245.33",
         "Host": "api.inews.qq.com",
         "Origin": "https://news.qq.com"
}
url="https://api.inews.qq.com/newsqa/v1/query/inner/publish/modules/list?modules=localCityNCOVDataList,diseaseh5Shelf"
resp=requests.post(url,headers=headers)
value_list=[]
for i in resp.json()['data']['diseaseh5Shelf']['areaTree'][0]['children']:
    area=i['name']
    local_confirm_add=i['today']['local_confirm_add']
    nowConfirm=i['total']['nowConfirm']
    confirm=i['total']['confirm']
    dead=i['total']['dead']
    heal=i['total']['heal']
    values=[area,local_confirm_add,nowConfirm,confirm,dead,heal]
    value_list.append(values)
column=['地区','新增确诊','现有确诊','累计确诊','累计死亡','累计治愈']
pd.DataFrame(value_list,columns=column).to_csv("全国疫情数据(含港澳台).csv",mode='w',encoding='gbk',index=False)
print("写入完毕")
resp.close()

写入完毕


## 代理ip

In [29]:
import requests
url="https://www.baidu.com"
proxies={"https":"https://91.67.201.74:8118"}
resp=requests.get(url,proxies=proxies,verify=False)
resp.close()

## 爬取网易云评论，直接拿加密后的表单数据

In [31]:
import requests
data={
    "params": "YvNQJHOM/X+JHkLAdAh9M/hFkosPOn0DorhOfiUg/wrPqY3cxDBYG5Fda9G507ukHAixs55NlpOltF//jDB9KEcitPMAM0P0gtZMqGzs9LosvfWRFteEV/dfaZFDfGcrRkbrYvdpYZ6vObaAnXU2LOsxqBKNHk2HfTWb1W0PBScTyagWTKQ7NGbYE5x9OEb36uOLoQ+TG0FTWgRFyOd2zpVxc7jkxZ0IAjxu3qF8nX49HtxizlX8KLqbYLNFMLEaiQ1cH3LE6y3F2QbVJasVI7HOYqL0A1nACPIgUMcnpqQ=",
    "encSecKey": "7f53cbaf3eda4e623f4e387247efa289df723d16b5d6517c7133e46e2d633b5918cfda3703f180bf552fbb5247097ee1bc6c74bbd00bccbe377fe70e330ba09367b3c04cee93ade18c4663bf2427108ea693f2d9314bc175b1ee947b4bd82cc7d6d13b073777bd5afdf533eae9f63c56eaf835cc515575f057cd871e2ed7992d"
}
resp = requests.post("https://music.163.com/weapi/comment/resource/comments/get?csrf_token=",data=data)
for i in resp.json()['data']['comments']:
    print(i['content'])
resp.close()

2022.6.7听闻华坪女高校长张桂梅“先生”事迹  专程来听一遍
2022.6.7听闻华女高校长张桂梅“先生”事迹  专程来听一遍
2021.6.7听闻华女高校长张桂梅“先生”事迹  专程来听一遍
好好听[多多大哭]
缅甸国歌[多多比耶]
清流
加油。三年后再见
张桂梅校长，希望您身体健康！
抖音里那只八哥儿
卧槽
哪有什么负重前行，只是有人替我们岁月静好
这首歌词一下回到乾隆年代了[呆]
看那个视频的时候，差点儿哭了
听的我哭了T﹏T。
小兄弟，只能说你比他们成熟，等他们长大了，就能知道自己之前多么愚蠢。
祝学姐学长们旗开得胜，前程似锦，金榜题名
我爱中华人民共和国
高考上岸！上本科！
加油
加油！


## 爬取网易云前3页评论，直接找到明文数据，计算加密后的表单数据

### 网易云明文加密方式
加密函数入口为d
```js
//a为数据
//a函数生成16位随机的字符串
function a(a) 
{
    var d, e, b = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789", c = "";
    for (d = 0; a > d; d += 1)//循环16次拿到16个字符
       e = Math.random() * b.length ,//一个浮点型伪随机数字，在0（包括 0）和1（不包括）之间。
       e = Math.floor(e),//取下整数
       c += b.charAt(e);
       return c
}
// a:数据，b:密钥
//b函数为AES加密过程
function b(a, b) 
{ 
    var c = CryptoJS.enc.Utf8.parse(b) //密钥
     , d = CryptoJS.enc.Utf8.parse("0102030405060708")//偏移量
     , e = CryptoJS.enc.Utf8.parse(a)//数据
     , f = CryptoJS.AES.encrypt(e, c, { iv: d, mode: CryptoJS.mode.CBC //加密模式
     });
     return f.toString()
}
function c(a, b, c) 
{
    var d, e;
    return setMaxDigits(131),
    d = new RSAKeyPair(b,"",c),
    e = encryptedString(d, a)
}
//d:明文数据的字符串形式,e:'010001',g:'0CoJUm6Qyw8W8jud',f='00e0b509f6259df8642dbc35662901477df22677ec152b5ff68ace615bb7b725152b3ab17a876aea8a5aa76d2e417629ec4ee341f56135fccf695280104e0312ecbda92557c93870114af6c9d05c4f7f0c3685b7a46bee255932575cce10b424d813cfe4875d3e82047b97ddef52741d546b8e289dc6935b3ece0462db0a22b8e7'
function d(d, e, f, g) 
{ 
    var h = {}, i = a(16); // i为16位字符串
    return h.encText = b(d, g),
    h.encText = b(h.encText, i), //params=h.encText
    h.encSecKey = c(i, e, f),//encseckey=h.encSecKey,这里我们处理是将i设为定值，从而他的encSecKey是定值
     h
}
```

In [6]:
import requests
from Crypto.Cipher import AES
from base64 import b64encode
import json 

i="HyVrziSyIouwjdhk"
g='0CoJUm6Qyw8W8jud'
data1={
    "csrf_token": "",
    "cursor": "-1",
    "offset": "0",
    "orderType": "1",
    "pageNo": "1",
    "pageSize": "20",
    "rid": "R_SO_4_1313584359",
    "threadId": "R_SO_4_1313584359"
}
url="https://music.163.com/weapi/comment/resource/comments/get?csrf_token="

def getParams(data2):
    first=enc_params(data2,g)
    second=enc_params(first,i)
    return second

def to_16(data4):
    pad=16-len(data4)%16
    data4+=pad*chr(pad)
    return data4
    
def enc_params(data3,key):
    iv="0102030405060708"
    data3=to_16(data3)
    aes=AES.new(key=key.encode('utf-8'),IV=iv.encode("utf-8"),mode=AES.MODE_CBC) #创建加密器
    bs=aes.encrypt(data3.encode('utf-8')) #加密的内容必须的是16的倍数且必须是字节，小于16个补够16个，不小于16个，则补够16的倍数个，"123456"=>""123456chr(10)chr(10)..."补够16个
    return str(b64encode(bs),'utf-8')#对byte类型进行base64编码返回一个byte对象，防止有些字节无法解码成字符串
    
def getEncSecKey():
    return "3712096275adc17448e4b611e346774802a0b8734837b85238e676ebe2458bfb3d5d985de1287bf30a6e2308ebaa5c1b25c3e794863155570ae4e0a8e126a03c38ad1064c5da2506838a76b7761749c76391b10c2b7820b67e85e06a0762589f5e62453e6bfd684a813355948579bbce999a81edb9d4967aac7bcdfccfb93f02"
    
data1['pageSize']=str(2*20)
data={"params":getParams(json.dumps(data1)),"encSecKey":getEncSecKey()} #json.dumps()将python对象编码成Json字符串
resp = requests.post(url,data=data)
with open("网易云评论.txt",mode='w',encoding='utf-8') as f:
    for i in resp.json()['data']['comments']:
        f.write(i['content']+'\n')
print("over")
resp.close()

over


# 多线程
进程：资源单位  
线程：执行单位  
线程池：一次性开辟一些线程，用户提交任务给线程池，线程任务调度交给线程池  
同理也进程池，与线程池类似  

## 线程池爬取北京新发地菜价爬取蔬菜前100页内容

In [3]:
import requests
from concurrent.futures import ThreadPoolExecutor
import csv
import time

f=open("新发地1-200页蔬菜数据.csv",'a',newline='')
f1=csv.writer(f)
url='http://www.xinfadi.com.cn/getPriceData.html'
params={
    "limit": 20,
    "current":"", 
    "pubDateStartTime":"",
    "pubDateEndTime": "",
    "prodPcatid": "",
    "prodCatid": "",
    "prodName": ""
}
    
def get_page(params):
    resp=requests.post(url,params=params)
    for i in resp.json()['list']:
        f1.writerow([i['prodName'],i['avgPrice'],i['highPrice'],i['lowPrice'],i['pubDate'],i['unitInfo']])
    resp.close()
    time.sleep(1)

f1.writerow(['prodName','avgPrice','highPrice','lowPrice','pubDate','unitInfo'])
with ThreadPoolExecutor(50) as t: #开辟一个拥有50个线程的线程池，将任务提交给他做任务
    for i in range(1,201):
        params['current']=i
        t.submit(get_page,params)
print("over")
f.close()

over
